# Setting up the experiment

In [1]:
#Dependency setup
!python -m pip install numpy matplotlib torch sklearn pandas

You should consider upgrading via the 'c:\Users\leon1\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# Centralized training

In [1]:

import datetime
import torch
import partition_scripts
from neural_nets import get_parameters, set_parameters, train, test, VGG7, Net, centralized_training

DATA_STORE = {
    "CIFAR10_IID": None,
    "CIFAR10_NonIID": None,
    "CIFAR100_IID": None,
    "CIFAR100_NonIID": None,
    "FedFaces_IID": None,
    "FedFaces_NonIID": None,

}

In [6]:
experiments = ["CIFAR10", "CIFAR100", "CelebA", "FedFaces"]
curr = experiments[3]
epochs = 20

match curr:
    case "CIFAR10":
        DATA_STORE["CIFAR10"] = partition_scripts.partition_CIFAR_IID(2)
        dataloaders, valloaders, testloaders = DATA_STORE["CIFAR10"]
        centralized_training(trainloader=dataloaders[0], valloader=valloaders[0], testloader=testloaders, epochs=epochs)
    case "CIFAR100":
        DATA_STORE["CIFAR100"] = partition_scripts.partition_CIFAR_IID(2, "CIFAR100")
        dataloaders, valloaders, testloaders = DATA_STORE["CIFAR100"]
        centralized_training(trainloader=dataloaders[0], valloader=valloaders[0], testloader=testloaders, epochs=epochs)
    case "CelebA":
        DATA_STORE["CelebA"] = partition_scripts.partition_CelebA_IID(2)
        dataloaders, valloaders, testloaders = DATA_STORE["CelebA"]
        centralized_training(trainloader=dataloaders[0], valloader=valloaders[0], testloader=testloaders, epochs=epochs)
    case "FedFaces":
        DATA_STORE["FedFaces"] = partition_scripts.partition_FedFaces_IID(2)
        dataloaders, valloaders, testloaders = DATA_STORE["FedFaces"]
        centralized_training(trainloader=dataloaders[0], valloader=valloaders[0], testloader=testloaders, epochs=epochs)
    case _:
        pass



Minority class count: 9266
Shape CIFAR IID: (64, 64)


RuntimeError: Input type (unsigned char) and bias type (float) should be the same

# Setting up a FLWR environment

In [3]:
import flwr as fl
import flwr.server.strategy as strategy

from flwr.common import Metrics


today = datetime.datetime.today()
fl.common.logger.configure(identifier="FL Paper Experiment", filename=f"log_FLWR_{today.timestamp()}.txt")

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

NUM_CLIENTS = 20
TRAINING_ROUNDS = 50

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = {"num_cpus": 1, "num_gpus": 0}

Training on cpu using PyTorch 2.2.0+cpu and Flower 1.7.0


Now, we'll set up the Client configurations

In [4]:
from typing import List, Tuple
from logging import DEBUG, INFO
from flwr.common.logger import log

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader, cid):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.cid = cid
        self.round = 0

    def get_parameters(self, config):
        # Return the current local parameters
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Train the local model after updating it with the given parameters
        # Return the parameters from the newly trained model, the length
        # of the training data, and a dict (empty in this case)
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs = 1)
        self.round+=1
        log(DEBUG, f"Client {self.cid} in round {self.round}")
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        # Perform the evaluation of the model after updating it with the given
        # parameters. Returns the loss as a float, the length of the validation
        # data, and a dict containing the accuracy
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return loss, len(self.valloader), {'accuracy': float(accuracy)}


def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

Then, setting up the strategies

#### FedAvg on CIFAR ####

In [5]:
# Create FedAvg strategy
fedAvg = fl.server.strategy.FedAvg(
    fraction_fit=1,  
    fraction_evaluate=0.5,  
    min_fit_clients=1,  
    min_evaluate_clients=1, 
    min_available_clients=1,
    evaluate_metrics_aggregation_fn=weighted_average,
)


# A couple of client_fns for using with Flower, one for each dataset experiment
def client_fn_CIFAR10_IID(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Create model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    trainloaders, valloaders,_ =  DATA_STORE["CIFAR10_IID"]
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader, cid).to_client()

def client_fn_CIFAR10_nonIID(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Create model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    trainloaders, valloaders,_ =  DATA_STORE["CIFAR10_NonIID"]
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader, cid).to_client()

A quick run to check if Flower is working ok

In [6]:
# Start simulation
run = False
if (run):
    fl.simulation.start_simulation(
        client_fn=client_fn_CIFAR10_IID,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=2),
        strategy=fedAvg,
        client_resources=client_resources,
    )

### FedExperiment Class ###
Ok, so now i'll encapsulate this code to reuse with different strategies and datasets

In [7]:
class FedExperiment():

    def __init__(self, client_fn,strategy, name="New experiment"):
        self.client_fn = client_fn
        self.strategy = strategy
        self.name = name

    def simulate_FL(self, rounds=1):
        log(INFO, "\n" + 10 * "========" + "\n" + self.name + " has started\n" + 10 * "========"  )
        metrics = fl.simulation.start_simulation(
                            client_fn=self.client_fn,
                            num_clients=NUM_CLIENTS,
                            config=fl.server.ServerConfig(num_rounds=rounds),
                            strategy=self.strategy,
                            client_resources=client_resources,
                        )
        log(INFO, "\n" + 10 * "========" + "\n" + self.name + " has ended\n" + 10 * "========"  )
        return metrics

In [9]:
DATA_STORE["CIFAR10_IID"]= partition_scripts.partition_CIFAR_IID(NUM_CLIENTS, "CIFAR10")
exp_CIFAR10_IID = FedExperiment(client_fn=client_fn_CIFAR10_IID, strategy=fedAvg, name="CIFAR 10 - IID Distribution")
metrics = exp_CIFAR10_IID.simulate_FL(rounds=5)
print(metrics)
DATA_STORE["CIFAR10_IID"]= None

INFO flwr 2024-02-23 13:45:51,322 | 527930960.py:9 | 
CIFAR 10 - IID Distribution has started
INFO flwr 2024-02-23 13:45:51,322 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)


Shape CIFAR IID: (32, 32, 3)


2024-02-23 13:45:55,826	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-02-23 13:45:57,577 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 12.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 2822028902.0, 'memory': 5644057806.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-02-23 13:45:57,577 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-02-23 13:45:57,577 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 0}
INFO flwr 2024-02-23 13:45:57,577 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 6 actors
INFO flwr 2024-02-23 13:45:57,577 | server.py:89 | Initializing global parameters
INFO flwr 2024-02-23 13:45:57,577 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2024-02-23 13:46:01,410 | server.py:280 | Received initial parameters from one random client
INFO flwr 2024-02-

History (loss, distributed):
	round 1: 0.03562954177856446
	round 2: 0.025969836235046385
	round 3: 0.02311199498176575
	round 4: 0.021931134033203126
	round 5: 0.020923221397399903
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.1348), (2, 0.40599999999999997), (3, 0.454), (4, 0.4932), (5, 0.5244000000000001)]}


In [9]:
DATA_STORE["CIFAR10_NonIID"] = partition_scripts.partition_CIFAR_nonIID(NUM_CLIENTS)
exp_CIFAR10_nonIID = FedExperiment(client_fn=client_fn_CIFAR10_nonIID, strategy=fedAvg, name="CIFAR 10 - nonIID Distribution")
metrics = exp_CIFAR10_nonIID.simulate_FL(rounds=5)
print(metrics)
DATA_STORE["CIFAR10_NonIID"] = partition_scripts.partition_CIFAR_nonIID(NUM_CLIENTS)

Shape CIFAR nonIID: (32, 32, 3)


INFO flwr 2024-02-23 14:24:39,699 | 527930960.py:9 | 
CIFAR 10 - nonIID Distribution has started
INFO flwr 2024-02-23 14:24:39,699 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2024-02-23 14:24:45,001	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-02-23 14:24:46,803 | app.py:213 | Flower VCE: Ray initialized with resources: {'memory': 5832088782.0, 'object_store_memory': 2916044390.0, 'node:127.0.0.1': 1.0, 'CPU': 12.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-02-23 14:24:46,818 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-02-23 14:24:46,819 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0}
INFO flwr 2024-02-23 14:24:46,819 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 12 actors
INFO flwr 2024-02-23 14:24:46,819 | server.py:89 | Initializing global paramet